In [1]:
import pandas as pd
import json
import os
from dataset import normalize_houses_folder, select_scenes, source_folder

In [2]:
source_folder = "3D-FUTURE-scene"
json_file_path = source_folder + '/train/train_set.json'

In [3]:
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

In [9]:
for key in data:
    print(key, len(data[key]))

categories 34
images 14761
annotations 73513


In [12]:
print("Categories")
print(data['categories'][0])

print("===========================")

print("Sample Annotation")
print(data['annotations'][0])

Categories
{'id': 1, 'super-category': 'Cabinet/Shelf/Desk', 'category': 'Children Cabinet'}
Sample Annotation
{'id': 1, 'image_id': 1, 'category_id': 33, 'segmentation': {'counts': [631406, 14, 1184, 20, 1176, 26, 1172, 30, 1169, 33, 1165, 37, 1162, 39, 1160, 41, 1157, 44, 1155, 46, 1154, 47, 1152, 49, 1150, 51, 1148, 52, 1148, 53, 1146, 55, 1145, 55, 1144, 57, 1143, 57, 1142, 59, 1141, 59, 38, 10, 1093, 59, 37, 13, 1090, 61, 34, 17, 1088, 60, 34, 19, 1087, 62, 31, 21, 9, 7, 1070, 62, 31, 21, 6, 13, 1067, 62, 30, 23, 3, 16, 1066, 61, 16, 1, 4, 1, 8, 25, 1, 19, 1064, 57, 1, 79, 1063, 137, 1063, 57, 1, 5, 3, 1, 5, 1, 4, 3, 1, 2, 3, 1, 3, 48, 1062, 62, 28, 49, 1061, 58, 1, 3, 28, 49, 1061, 58, 1, 3, 28, 50, 1060, 61, 30, 49, 1061, 60, 30, 49, 1061, 60, 30, 49, 1061, 59, 31, 49, 1062, 58, 32, 48, 1062, 58, 33, 47, 1062, 57, 35, 46, 1063, 56, 36, 17, 1, 27, 1063, 55, 38, 15, 3, 25, 1065, 54, 40, 11, 5, 25, 1066, 52, 44, 8, 6, 23, 1067, 51, 46, 7, 6, 23, 1068, 50, 46, 7, 7, 21, 1070, 48, 47

In [13]:
# Create Pandas tables
# Categories
categories_data = data['categories']
df_categories = pd.DataFrame(categories_data)

df_categories.head()

,id,super-category,category
0,1,Cabinet/Shelf/Desk,Children Cabinet
1,2,Cabinet/Shelf/Desk,Nightstand
2,3,Cabinet/Shelf/Desk,Bookcase / jewelry Armoire
3,4,Cabinet/Shelf/Desk,Wardrobe
4,5,Cabinet/Shelf/Desk,Coffee Table


In [15]:
# Images
images_data = data['images']
df_images = pd.DataFrame(images_data)

df_images.head()

,id,width,height,file_name
0,1,1200,1200,0000000
1,2,1200,1200,0000001
2,3,1200,1200,0000002
3,4,1200,1200,0000003
4,5,1200,1200,0000004


In [14]:
# Annotations
annotations_data = [{k: v for k, v in obj.items() if k != 'segmentation'} for obj in data['annotations']]
df_annotations = pd.DataFrame(annotations_data)

df_annotations.head()

,id,image_id,category_id,area,bbox,model_id,texture_id,fov,style,theme,material,pose,occlusion
0,1,1,33,13073,"[526.0, 56.0, 238.0, 305.0]",da64b6ef-d704-404b-a0e3-866d0e091362,da64b6ef-d704-404b-a0e3-866d0e091362,0.958973,Modern,None,None,"{'translation': [0.18415942747456682, 0.767061...",NO
1,2,1,2,8014,"[1140.0, 795.0, 60.0, 149.0]",4d5ff861-d5ce-4f10-90c2-c064b1845199,4d5ff861-d5ce-4f10-90c2-c064b1845199,,Korean,Smooth Net,Composite Board,"{'translation': None, 'rotation': None, 'euler...",N/A
2,3,1,23,24436,"[0.0, 603.0, 249.0, 310.0]",a939be24-9c3d-44f9-b713-e74ddee892df,a939be24-9c3d-44f9-b713-e74ddee892df,0.958973,Light Luxury,Gold Foil,Composite Board,"{'translation': [-1.9192422163710339, -1.20000...",N/A
3,4,1,4,211496,"[173.0, 287.0, 434.0, 623.0]",1d9f21cd-359f-4520-ad98-467e86526b35,1d9f21cd-359f-4520-ad98-467e86526b35,0.958973,Korean,Smooth Net,Composite Board,"{'translation': [-0.8610572589744727, -1.20000...",NO
4,5,1,13,235956,"[414.0, 553.0, 740.0, 499.0]",433123cb-172e-49ed-be20-24ca9159b0e4,433123cb-172e-49ed-be20-24ca9159b0e4,0.958973,Mediterranean,Striped Grid,Rough Cloth,"{'translation': [0.7791540350407389, -1.200000...",NO


In [19]:
annotations_joined = df_annotations.merge(df_images, left_on="image_id", right_on="id")
annotations_joined.rename(columns={'id_x': 'annotation_id'}, inplace=True)
annotations_joined = annotations_joined.drop('id_y', axis=1)
annotations_joined.head()

,annotation_id,image_id,category_id,area,bbox,model_id,texture_id,fov,style,theme,material,pose,occlusion,width,height,file_name
0,1,1,33,13073,"[526.0, 56.0, 238.0, 305.0]",da64b6ef-d704-404b-a0e3-866d0e091362,da64b6ef-d704-404b-a0e3-866d0e091362,0.958973,Modern,None,None,"{'translation': [0.18415942747456682, 0.767061...",NO,1200,1200,0000000
1,2,1,2,8014,"[1140.0, 795.0, 60.0, 149.0]",4d5ff861-d5ce-4f10-90c2-c064b1845199,4d5ff861-d5ce-4f10-90c2-c064b1845199,,Korean,Smooth Net,Composite Board,"{'translation': None, 'rotation': None, 'euler...",N/A,1200,1200,0000000
2,3,1,23,24436,"[0.0, 603.0, 249.0, 310.0]",a939be24-9c3d-44f9-b713-e74ddee892df,a939be24-9c3d-44f9-b713-e74ddee892df,0.958973,Light Luxury,Gold Foil,Composite Board,"{'translation': [-1.9192422163710339, -1.20000...",N/A,1200,1200,0000000
3,4,1,4,211496,"[173.0, 287.0, 434.0, 623.0]",1d9f21cd-359f-4520-ad98-467e86526b35,1d9f21cd-359f-4520-ad98-467e86526b35,0.958973,Korean,Smooth Net,Composite Board,"{'translation': [-0.8610572589744727, -1.20000...",NO,1200,1200,0000000
4,5,1,13,235956,"[414.0, 553.0, 740.0, 499.0]",433123cb-172e-49ed-be20-24ca9159b0e4,433123cb-172e-49ed-be20-24ca9159b0e4,0.958973,Mediterranean,Striped Grid,Rough Cloth,"{'translation': [0.7791540350407389, -1.200000...",NO,1200,1200,0000000


In [ ]:
# Concatenate room captions
# For each image
# ~~ BLIP descriptions ~~ . The room has {style} {category}, {style} {category}, ...
# e.g. The room has Mediterranean style Coffee table, Korean style Sofa